In [3]:
import nfl_data_py as nfl
import pandas as pd
import os
import urllib.request
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnnotationBbox
from matplotlib.offsetbox import OffsetImage

from PIL import Image
import numpy as np
from io import BytesIO
import requests
# ML Libraryies
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import hvplot.pandas
import plotly.express as px

# Graphing 
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.pyplot import figure

In [4]:
#Import Data
nfl_data = nfl.import_schedules([2023])
original_df = nfl_data.groupby('week')
# Get data from games
weekly_nfl = nfl.import_weekly_data([2017,2018,2019,2021,2022,2023])
nfl_this_season = nfl.import_weekly_data([2023])
weekly_nfl.dtypes

Downcasting floats.
Downcasting floats.


player_id                       object
player_name                     object
player_display_name             object
position                        object
position_group                  object
headshot_url                    object
recent_team                     object
season                           int32
week                             int32
season_type                     object
completions                      int32
attempts                         int32
passing_yards                  float32
passing_tds                      int32
interceptions                  float32
sacks                          float32
sack_yards                     float32
sack_fumbles                     int32
sack_fumbles_lost                int32
passing_air_yards              float32
passing_yards_after_catch      float32
passing_first_downs            float32
passing_epa                    float32
passing_2pt_conversions          int32
pacr                           float32
dakota                   

In [5]:
# columns for receiving stats per game

columns = [ 'week','player_display_name','recent_team','opponent_team','receptions',
           'targets', 'receiving_yards','receiving_tds','receiving_epa','receiving_air_yards', 'receiving_yards_after_catch',
           'target_share', 'wopr', 'pacr'
           ]

In [6]:
#create Df
games_df = weekly_nfl[columns]
games_df.head()

,week,player_display_name,recent_team,opponent_team,receptions,targets,receiving_yards,receiving_tds,receiving_epa,receiving_air_yards,receiving_yards_after_catch,target_share,wopr,pacr
0,1,Tom Brady,NE,NaN,0,0,0.0,0,NaN,0.0,0.0,NaN,NaN,0.470899
1,2,Tom Brady,NE,NaN,0,0,0.0,0,NaN,0.0,0.0,NaN,NaN,1.590747
2,3,Tom Brady,NE,NaN,0,0,0.0,0,NaN,0.0,0.0,NaN,NaN,0.910843
3,4,Tom Brady,NE,NaN,0,0,0.0,0,NaN,0.0,0.0,NaN,NaN,0.900293
4,5,Tom Brady,NE,NaN,0,0,0.0,0,NaN,0.0,0.0,NaN,NaN,1.221774


In [7]:
# Groupby opponent and week offense stats
team_by_week = games_df.groupby(['week', 'opponent_team','recent_team'])
team_by_week.head()
team_stats_per_game = team_by_week.sum()
team_stats_per_game.index.set_names(['week', 'team_defense','recent_team'], inplace=True)
team_stats_per_game = team_stats_per_game.reset_index()
team_stats_per_game.head()

,week,team_defense,recent_team,receptions,targets,receiving_yards,receiving_tds,receiving_epa,receiving_air_yards,receiving_yards_after_catch,target_share,wopr,pacr
0,1,ARI,WAS,19,29,202.0,1,1.409773,208.0,68.0,1.0,2.2,0.885965
1,1,ATL,CAR,20,31,146.0,1,-3.103000,228.0,92.0,1.0,2.2,0.519573
2,1,BAL,HOU,28,43,242.0,0,2.796677,260.0,133.0,1.0,2.2,0.930769
3,1,BUF,NYJ,14,18,140.0,1,3.475710,75.0,92.0,1.0,2.2,1.458333
4,1,CAR,ATL,15,18,115.0,1,0.718799,63.0,75.0,1.0,2.2,1.825397


In [8]:
# add _def to columns
columns_to_suffix_def = [
    'receptions',
    'targets',
    'receiving_yards',
    'receiving_tds',
    'receiving_epa',
    'receiving_air_yards',
    'receiving_yards_after_catch',
    'target_share',
    'fantasy_points_ppr',
    'wopr',
    'pacr'
]

# Add "_def" to specific columns
team_stats_per_game = team_stats_per_game.rename(columns={col: col + '_def' for col in columns_to_suffix_def})
team_stats_per_game.head()

,week,team_defense,recent_team,receptions_def,targets_def,receiving_yards_def,receiving_tds_def,receiving_epa_def,receiving_air_yards_def,receiving_yards_after_catch_def,target_share_def,wopr_def,pacr_def
0,1,ARI,WAS,19,29,202.0,1,1.409773,208.0,68.0,1.0,2.2,0.885965
1,1,ATL,CAR,20,31,146.0,1,-3.103000,228.0,92.0,1.0,2.2,0.519573
2,1,BAL,HOU,28,43,242.0,0,2.796677,260.0,133.0,1.0,2.2,0.930769
3,1,BUF,NYJ,14,18,140.0,1,3.475710,75.0,92.0,1.0,2.2,1.458333
4,1,CAR,ATL,15,18,115.0,1,0.718799,63.0,75.0,1.0,2.2,1.825397


In [9]:
# create receiver stats per player per week vs opp
# Create a DF for WR's and TE's and RB's vs the pass
wr_df_two = weekly_nfl[(weekly_nfl["position"] == "WR") | (weekly_nfl["position"] == "TE") | (weekly_nfl["position"] == "RB")]
wr_df = wr_df_two[columns]
#wr_df= wr_df[wr_df['fantasy_points_ppr'] != 0]
wr_df.head()


,week,player_display_name,recent_team,opponent_team,receptions,targets,receiving_yards,receiving_tds,receiving_epa,receiving_air_yards,receiving_yards_after_catch,target_share,wopr,pacr
57,1,Antonio Gates,LAC,NaN,2,3,17.0,0,0.303061,16.0,8.0,0.090909,0.192929,NaN
58,2,Antonio Gates,LAC,NaN,2,4,11.0,1,0.946824,28.0,0.0,0.102564,0.219839,NaN
59,3,Antonio Gates,LAC,NaN,2,5,30.0,0,-5.984768,37.0,11.0,0.128205,0.254269,NaN
60,4,Antonio Gates,LAC,NaN,3,5,18.0,0,-0.309226,16.0,8.0,0.135135,0.245615,NaN
61,5,Antonio Gates,LAC,NaN,2,3,23.0,0,1.059868,24.0,7.0,0.071429,0.167358,NaN


In [10]:
#groupby team and opponent and player
wr_v_d = wr_df.groupby(['opponent_team','recent_team', 'player_display_name','week']).mean()
wr_v_d.head()

receptions  targets  \
opponent_team recent_team player_display_name week                        
ARI           BAL         Gus Edwards         8            2.0      2.0   
                          Isaiah Likely       8            1.0      1.0   
                          Justice Hill        8            4.0      4.0   
                          Mark Andrews        8            4.0      5.0   
                          Nelson Agholor      8            0.0      1.0   

                                                    receiving_yards  \
opponent_team recent_team player_display_name week                    
ARI           BAL         Gus Edwards         8                14.0   
                          Isaiah Likely       8                10.0   
                          Justice Hill        8                40.0   
                          Mark Andrews        8                40.0   
                          Nelson Agholor      8                 0.0   

                                                    receiving_tds  \
opponent_team recent_team player_display_name week                  
ARI           BAL         Gus Edwards         8               0.0   
                          Isaiah Likely       8               0.0   
                          Justice Hill        8               0.0   
                          Mark Andrews        8               1.0   
                          Nelson Agholor      8               0.0   

                                                    receiving_epa  \
opponent_team recent_team player_display_name week                  
ARI           BAL         Gus Edwards         8          0.714885   
                          Isaiah Likely       8          0.611771   
                          Justice Hill        8          0.888634   
                          Mark Andrews        8          6.058561   
                          Nelson Agholor      8         -0.741357   

                                                    receiving_air_yards  \
opponent_team recent_team player_display_name week                        
ARI           BAL         Gus Edwards         8                     7.0   
                          Isaiah Likely       8                    10.0   
                          Justice Hill        8                    -4.0   
                          Mark Andrews        8                    24.0   
                          Nelson Agholor      8                    13.0   

                                                    receiving_yards_after_catch  \
opponent_team recent_team player_display_name week                                
ARI           BAL         Gus Edwards         8                             7.0   
                          Isaiah Likely       8                             0.0   
                          Justice Hill        8                            44.0   
                          Mark Andrews        8                            19.0   
                          Nelson Agholor      8                             0.0   

                                                    target_share      wopr  \
opponent_team recent_team player_display_name week                           
ARI           BAL         Gus Edwards         8         0.076923  0.146998   
                          Isaiah Likely       8         0.038462  0.102854   
                          Justice Hill        8         0.153846  0.212705   
                          Mark Andrews        8         0.192308  0.396849   
                          Nelson Agholor      8         0.038462  0.116402   

                                                    pacr  
opponent_team recent_team player_display_name week        
ARI           BAL         Gus Edwards         8      NaN  
                          Isaiah Likely       8      NaN  
                          Justice Hill        8      NaN  
                          Mark Andrews        8      NaN  
                          Nelson Agholor      8    

In [11]:
#reset index
wr_v_d_df = wr_v_d.reset_index()
wr_v_d_df.head()

,opponent_team,recent_team,player_display_name,week,receptions,targets,receiving_yards,receiving_tds,receiving_epa,receiving_air_yards,receiving_yards_after_catch,target_share,wopr,pacr
0,ARI,BAL,Gus Edwards,8,2.0,2.0,14.0,0.0,0.714885,7.0,7.0,0.076923,0.146998,NaN
1,ARI,BAL,Isaiah Likely,8,1.0,1.0,10.0,0.0,0.611771,10.0,0.0,0.038462,0.102854,NaN
2,ARI,BAL,Justice Hill,8,4.0,4.0,40.0,0.0,0.888634,-4.0,44.0,0.153846,0.212705,NaN
3,ARI,BAL,Mark Andrews,8,4.0,5.0,40.0,1.0,6.058561,24.0,19.0,0.192308,0.396849,NaN
4,ARI,BAL,Nelson Agholor,8,0.0,1.0,0.0,0.0,-0.741357,13.0,0.0,0.038462,0.116402,NaN


In [12]:
#change opponent team name to team_defense
wr_v_d_df = wr_v_d_df.rename(columns={'opponent_team': 'team_defense'})


In [13]:
# merge players stats per game vs the defense stats that game

test_data_df = wr_v_d_df.merge(team_stats_per_game, on=['week', 'team_defense', 'recent_team'])
test_data_df = test_data_df.fillna(0)


In [14]:
test_data_df.head(10)

,team_defense,recent_team,player_display_name,week,receptions,targets,receiving_yards,receiving_tds,receiving_epa,receiving_air_yards,...,receptions_def,targets_def,receiving_yards_def,receiving_tds_def,receiving_epa_def,receiving_air_yards_def,receiving_yards_after_catch_def,target_share_def,wopr_def,pacr_def
0,ARI,BAL,Gus Edwards,8,2.0,2.0,14.0,0.0,0.714885,7.0,...,18,26,157.0,1,0.411072,155.0,85.0,1.0,2.2,0.981250
1,ARI,BAL,Isaiah Likely,8,1.0,1.0,10.0,0.0,0.611771,10.0,...,18,26,157.0,1,0.411072,155.0,85.0,1.0,2.2,0.981250
2,ARI,BAL,Justice Hill,8,4.0,4.0,40.0,0.0,0.888634,-4.0,...,18,26,157.0,1,0.411072,155.0,85.0,1.0,2.2,0.981250
3,ARI,BAL,Mark Andrews,8,4.0,5.0,40.0,1.0,6.058561,24.0,...,18,26,157.0,1,0.411072,155.0,85.0,1.0,2.2,0.981250
4,ARI,BAL,Nelson Agholor,8,0.0,1.0,0.0,0.0,-0.741357,13.0,...,18,26,157.0,1,0.411072,155.0,85.0,1.0,2.2,0.981250
5,ARI,BAL,Odell Beckham,8,0.0,4.0,0.0,0.0,-4.786708,47.0,...,18,26,157.0,1,0.411072,155.0,85.0,1.0,2.2,0.981250
6,ARI,BAL,Rashod Bateman,8,2.0,2.0,34.0,0.0,1.465703,34.0,...,18,26,157.0,1,0.411072,155.0,85.0,1.0,2.2,0.981250
7,ARI,BAL,Zay Flowers,8,5.0,7.0,19.0,0.0,-3.800416,24.0,...,18,26,157.0,1,0.411072,155.0,85.0,1.0,2.2,0.981250
8,ARI,CIN,Andrei Iosivas,5,1.0,2.0,9.0,0.0,-0.727112,25.0,...,36,44,317.0,3,16.416182,298.0,155.0,1.0,2.2,1.056667
9,ARI,CIN,Chase Brown,5,1.0,1.0,2.0,0.0,-0.520796,-5.0,...,36,44,317.0,3,16.416182,298.0,155.0,1.0,2.2,1.056667


In [15]:
# create test data 

x_cols_train = ['receiving_epa',
                 'target_share',
                'receiving_epa_def', 'targets_def'
                
    
]


y_cols_train = ['receiving_yards','receiving_tds','receptions']

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# model
# Split data into features (X) and target variable (y)
X = test_data_df[x_cols_train]
y = test_data_df[y_cols_train]

# Encode categorical variables using one-hot encoding if needed

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Create a Linear Regression model
model = LinearRegression()

# Train the model
model.fit(X_train, y_train)

# Make predictions
predictions = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = mean_squared_error(y_test, predictions, squared=False)
r2 = r2_score(y_test, predictions)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R2): {r2}")

Mean Absolute Error (MAE): 3.436994464728049
Mean Squared Error (MSE): 60.46188145230381
Root Mean Squared Error (RMSE): 4.914358118670566
R-squared (R2): 0.6212123802364534


In [17]:
#create Df
games_df_test = nfl_this_season[columns]
games_df_test.head()

,week,player_display_name,recent_team,opponent_team,receptions,targets,receiving_yards,receiving_tds,receiving_epa,receiving_air_yards,receiving_yards_after_catch,target_share,wopr,pacr
0,1,Aaron Rodgers,NYJ,BUF,0,0,0.0,0,NaN,0.0,0.0,NaN,NaN,0.000000
1,4,Marcedes Lewis,CHI,DEN,1,1,8.0,0,0.483465,-3.0,11.0,0.031250,0.038197,NaN
2,7,Marcedes Lewis,CHI,LV,1,1,16.0,0,1.437224,5.0,11.0,0.034483,0.117762,NaN
3,1,Matthew Stafford,LA,SEA,0,0,0.0,0,NaN,0.0,0.0,NaN,NaN,0.816626
4,2,Matthew Stafford,LA,SF,0,0,0.0,0,NaN,0.0,0.0,NaN,NaN,0.845730


In [18]:
# Groupby opponent and week offense stats
team_by_week_test = games_df_test.groupby(['week', 'opponent_team','recent_team'])
#team_by_week.head()
team_stats_per_game_test = team_by_week_test.sum()
team_stats_per_game_test.index.set_names(['week', 'team_defense','recent_team'], inplace=True)
team_stats_per_game_test = team_stats_per_game_test.reset_index()
team_stats_per_game_test.head()

,week,team_defense,recent_team,receptions,targets,receiving_yards,receiving_tds,receiving_epa,receiving_air_yards,receiving_yards_after_catch,target_share,wopr,pacr
0,1,ARI,WAS,19,29,202.0,1,1.409773,208.0,68.0,1.0,2.2,0.885965
1,1,ATL,CAR,20,31,146.0,1,-3.103000,228.0,92.0,1.0,2.2,0.519573
2,1,BAL,HOU,28,43,242.0,0,2.796677,260.0,133.0,1.0,2.2,0.930769
3,1,BUF,NYJ,14,18,140.0,1,3.475710,75.0,92.0,1.0,2.2,1.458333
4,1,CAR,ATL,15,18,115.0,1,0.718799,63.0,75.0,1.0,2.2,1.825397


In [19]:
# Add "_def" to specific columns
team_stats_per_game_test = team_stats_per_game_test.rename(columns={col: col + '_def' for col in columns_to_suffix_def})
team_stats_per_game_test.head()

,week,team_defense,recent_team,receptions_def,targets_def,receiving_yards_def,receiving_tds_def,receiving_epa_def,receiving_air_yards_def,receiving_yards_after_catch_def,target_share_def,wopr_def,pacr_def
0,1,ARI,WAS,19,29,202.0,1,1.409773,208.0,68.0,1.0,2.2,0.885965
1,1,ATL,CAR,20,31,146.0,1,-3.103000,228.0,92.0,1.0,2.2,0.519573
2,1,BAL,HOU,28,43,242.0,0,2.796677,260.0,133.0,1.0,2.2,0.930769
3,1,BUF,NYJ,14,18,140.0,1,3.475710,75.0,92.0,1.0,2.2,1.458333
4,1,CAR,ATL,15,18,115.0,1,0.718799,63.0,75.0,1.0,2.2,1.825397


In [20]:
# Create a DF for WR's and TE's and RB's vs the pass
wr_df_two_test = nfl_this_season[(nfl_this_season["position"] == "WR") | (nfl_this_season["position"] == "TE") | (nfl_this_season["position"] == "RB")]
wr_df_test = wr_df_two_test[columns]
#wr_df= wr_df[wr_df['fantasy_points_ppr'] != 0]
wr_df_test.head()

,week,player_display_name,recent_team,opponent_team,receptions,targets,receiving_yards,receiving_tds,receiving_epa,receiving_air_yards,receiving_yards_after_catch,target_share,wopr,pacr
1,4,Marcedes Lewis,CHI,DEN,1,1,8.0,0,0.483465,-3.0,11.0,0.031250,0.038197,NaN
2,7,Marcedes Lewis,CHI,LV,1,1,16.0,0,1.437224,5.0,11.0,0.034483,0.117762,NaN
13,3,Jimmy Graham,NO,GB,1,1,8.0,1,2.708027,8.0,0.0,0.029412,0.064481,NaN
14,5,Jimmy Graham,NO,NE,0,1,0.0,0,-1.904473,6.0,0.0,0.040000,0.077500,NaN
15,7,Julio Jones,PHI,MIA,1,1,3.0,0,-0.153665,2.0,1.0,0.033333,0.056364,NaN


In [21]:
#groupby team and opponent and player
wr_v_d_test = wr_df_test.groupby(['opponent_team','recent_team', 'player_display_name','week']).mean()
#reset index
wr_v_d_df_test = wr_v_d_test.reset_index()
wr_v_d_df_test = wr_v_d_df_test.rename(columns={'opponent_team': 'team_defense'})
wr_v_d_df_test.head()


,team_defense,recent_team,player_display_name,week,receptions,targets,receiving_yards,receiving_tds,receiving_epa,receiving_air_yards,receiving_yards_after_catch,target_share,wopr,pacr
0,ARI,BAL,Gus Edwards,8,2.0,2.0,14.0,0.0,0.714885,7.0,7.0,0.076923,0.146998,NaN
1,ARI,BAL,Isaiah Likely,8,1.0,1.0,10.0,0.0,0.611771,10.0,0.0,0.038462,0.102854,NaN
2,ARI,BAL,Justice Hill,8,4.0,4.0,40.0,0.0,0.888634,-4.0,44.0,0.153846,0.212705,NaN
3,ARI,BAL,Mark Andrews,8,4.0,5.0,40.0,1.0,6.058561,24.0,19.0,0.192308,0.396849,NaN
4,ARI,BAL,Nelson Agholor,8,0.0,1.0,0.0,0.0,-0.741357,13.0,0.0,0.038462,0.116402,NaN


In [22]:
# merge players stats per game vs the defense stats that game

test_data_df_test = wr_v_d_df_test.merge(team_stats_per_game_test, on=['week', 'team_defense', 'recent_team'])
test_data_df_test = test_data_df_test.fillna(0)

In [23]:
# Create the averages for receivers
## First Create a mean of Each Players stats 

player_mean = wr_df_test.groupby('player_display_name').mean()
#remove week
player_mean.drop("week", axis=1, inplace=True)


player_mean.head()


,receptions,targets,receiving_yards,receiving_tds,receiving_epa,receiving_air_yards,receiving_yards_after_catch,target_share,wopr,pacr
player_display_name,,,,,,,,,,
A.J. Brown,7.444444,10.222222,111.666664,0.666667,6.273258,132.888885,35.111111,0.332239,0.836451,NaN
A.J. Dillon,1.375000,1.750000,12.875000,0.000000,0.432136,-1.500000,14.625000,0.063215,0.089132,NaN
Aaron Jones,2.800000,4.400000,29.400000,0.200000,1.550219,-5.200000,32.400002,0.145644,0.198615,NaN
Adam Thielen,7.555556,9.555556,72.444443,0.444444,4.292795,64.222221,27.222221,0.268055,0.595851,NaN
Adam Trautman,1.571429,2.571429,9.857142,0.142857,-1.059541,16.857143,5.000000,0.092390,0.223966,NaN


In [24]:
# Add recent_team to player_mean
team_cols = ['player_display_name', 'recent_team']
team_player = test_data_df_test[team_cols]
# merge to player_mean
player_mean_df = player_mean.merge(team_player, on='player_display_name')
player_mean_df = player_mean_df.drop_duplicates()
player_mean_df = player_mean_df.fillna(0)
player_mean_df.head()

,player_display_name,receptions,targets,receiving_yards,receiving_tds,receiving_epa,receiving_air_yards,receiving_yards_after_catch,target_share,wopr,pacr,recent_team
0,A.J. Brown,7.444444,10.222222,111.666664,0.666667,6.273258,132.888885,35.111111,0.332239,0.836451,0.0,PHI
9,A.J. Dillon,1.375000,1.750000,12.875000,0.000000,0.432136,-1.500000,14.625000,0.063215,0.089132,0.0,GB
17,Aaron Jones,2.800000,4.400000,29.400000,0.200000,1.550219,-5.200000,32.400002,0.145644,0.198615,0.0,GB
22,Adam Thielen,7.555556,9.555556,72.444443,0.444444,4.292795,64.222221,27.222221,0.268055,0.595851,0.0,CAR
31,Adam Trautman,1.571429,2.571429,9.857142,0.142857,-1.059541,16.857143,5.000000,0.092390,0.223966,0.0,DEN


In [25]:
# now get teams average for these stats
def_mean = wr_df_test.groupby(['week', 'opponent_team']).sum()

#get mean of team vs receiving stats
total_sum_def = def_mean.reset_index()
team_sum_stats = total_sum_def.groupby('opponent_team').mean()
team_sum_stats.drop("week", axis=1, inplace=True)

In [26]:
# Add "_def" to specific columns
team_sum_stats = team_sum_stats.rename(columns={col: col + '_def' for col in columns_to_suffix_def})
team_sum_stats = team_sum_stats.reset_index()
team_sum_stats.head()

,opponent_team,receptions_def,targets_def,receiving_yards_def,receiving_tds_def,receiving_epa_def,receiving_air_yards_def,receiving_yards_after_catch_def,target_share_def,wopr_def,pacr_def
0,ARI,21.666667,29.333333,243.222229,1.555556,8.901278,220.666672,100.666664,0.994709,2.190345,0.0
1,ATL,20.444444,30.222222,210.666672,1.888889,5.907104,245.111115,95.333336,1.000000,2.200000,0.0
2,BAL,22.333333,35.333333,203.888885,0.666667,-0.813307,265.888885,95.000000,0.997416,2.196423,0.0
3,BUF,22.555556,31.333333,236.222229,1.111111,5.022120,199.444443,110.000000,0.996732,2.187611,0.0
4,CAR,18.111111,26.444444,190.222229,1.222222,4.258365,189.111115,91.888885,0.982662,2.181072,0.0


In [27]:
# create DF for upcoming week
# create stats to show upcpoming games
grouped = nfl_data.groupby('week')

# Select the group corresponding to week 6
week_6_data = grouped.get_group(10)
get_teams =['home_team','away_team']
schedule_week_df = week_6_data[get_teams]
schedule_week_df.reset_index(drop=True, inplace=True)
schedule_week_df.head()

,home_team,away_team
0,CHI,CAR
1,NE,IND
2,BAL,CLE
3,CIN,HOU
4,JAX,SF


In [28]:
# rename cols 
schedule_week_df = schedule_week_df.rename(columns={'home_team': 'recent_team', 'away_team': 'opponent_team'})
schedule_week_df.head()

,recent_team,opponent_team
0,CHI,CAR
1,NE,IND
2,BAL,CLE
3,CIN,HOU
4,JAX,SF


In [29]:
# Create a reversed DataFrame
reversed_df = pd.DataFrame({'recent_team': schedule_week_df['opponent_team'], 'opponent_team': schedule_week_df['recent_team'],})

# Concatenate the original and reversed DataFrames
final_week_df = pd.concat([schedule_week_df, reversed_df], ignore_index=True)

final_week_df.head(3)

,recent_team,opponent_team
0,CHI,CAR
1,NE,IND
2,BAL,CLE


In [30]:
# merge team_sum_stats with final_week_df

testing_def = final_week_df.merge(team_sum_stats, on='opponent_team')
testing_def.head()

,recent_team,opponent_team,receptions_def,targets_def,receiving_yards_def,receiving_tds_def,receiving_epa_def,receiving_air_yards_def,receiving_yards_after_catch_def,target_share_def,wopr_def,pacr_def
0,CHI,CAR,18.111111,26.444444,190.222229,1.222222,4.258365,189.111115,91.888885,0.982662,2.181072,0.000000
1,NE,IND,22.777778,34.000000,254.111115,1.111111,4.786409,270.777771,104.222221,0.996169,2.194049,0.000000
2,BAL,CLE,14.500000,25.250000,164.125000,1.000000,-2.299216,194.125000,91.625000,0.987847,2.177358,0.000000
3,CIN,HOU,23.250000,31.875000,243.750000,0.875000,6.129324,234.375000,114.375000,0.974971,2.146206,0.229167
4,JAX,SF,26.500000,38.000000,244.375000,1.250000,-0.287219,273.750000,111.625000,1.000000,2.200000,0.000000


In [31]:
# merge player_mean_df with testing_def

final_test_df = testing_def.merge(player_mean_df, on='recent_team')
# Replace NaN values with 0
final_test_df = final_test_df.fillna(0)


In [32]:
final_test_df.head(16)

,recent_team,opponent_team,receptions_def,targets_def,receiving_yards_def,receiving_tds_def,receiving_epa_def,receiving_air_yards_def,receiving_yards_after_catch_def,target_share_def,...,receptions,targets,receiving_yards,receiving_tds,receiving_epa,receiving_air_yards,receiving_yards_after_catch,target_share,wopr,pacr
0,CHI,CAR,18.111111,26.444444,190.222229,1.222222,4.258365,189.111115,91.888885,0.982662,...,1.250000,4.000000,16.500000,0.250000,-2.306996,46.000000,3.000000,0.146681,0.352635,0.0
1,CHI,CAR,18.111111,26.444444,190.222229,1.222222,4.258365,189.111115,91.888885,0.982662,...,5.111111,6.333333,46.555557,0.555556,2.006372,35.555557,21.777779,0.212410,0.440774,0.0
2,CHI,CAR,18.111111,26.444444,190.222229,1.222222,4.258365,189.111115,91.888885,0.982662,...,1.500000,2.166667,9.166667,0.166667,0.146554,-0.666667,9.000000,0.082794,0.108680,0.0
3,CHI,CAR,18.111111,26.444444,190.222229,1.222222,4.258365,189.111115,91.888885,0.982662,...,5.200000,7.100000,79.300003,0.500000,3.250134,80.400002,34.700001,0.256620,0.681005,0.0
4,CHI,CAR,18.111111,26.444444,190.222229,1.222222,4.258365,189.111115,91.888885,0.982662,...,2.444444,4.222222,35.666668,0.111111,-0.085587,43.666668,15.666667,0.143415,0.399734,0.0
5,CHI,CAR,18.111111,26.444444,190.222229,1.222222,4.258365,189.111115,91.888885,0.982662,...,1.400000,2.400000,9.800000,0.000000,-1.045828,-2.800000,13.800000,0.078529,0.068681,0.0
6,CHI,CAR,18.111111,26.444444,190.222229,1.222222,4.258365,189.111115,91.888885,0.982662,...,0.666667,1.000000,8.333333,0.000000,0.210837,7.000000,0.666667,0.034167,0.072014,0.0
7,CHI,CAR,18.111111,26.444444,190.222229,1.222222,4.258365,189.111115,91.888885,0.982662,...,2.000000,3.600000,16.600000,0.200000,-2.240784,-1.400000,18.200001,0.124456,0.180172,0.0
8,CHI,CAR,18.111111,26.444444,190.222229,1.222222,4.258365,189.111115,91.888885,0.982662,...,1.000000,1.000000,12.000000,0.000000,0.960344,1.000000,11.000000,0.032866,0.077980,0.0
9,CHI,CAR,18.111111,26.444444,190.222229,1.222222,4.258365,189.111115,91.888885,0.982662,...,1.166667,1.500000,9.666667,0.000000,0.487377,7.333333,6.000000,0.053514,0.107547,0.0


In [33]:
# Now, use the trained model to predict rb_yards for df_test
X_test = final_test_df[x_cols_train]
y_pred = model.predict(X_test)

# The predictions are stored in y_pred
print("Predicted Scores:")
#print(y_pred)

Predicted Scores:


In [34]:
# add predicted scores into df: 
predict_cols = ['recent_team','opponent_team','player_display_name']
df_predictions = final_test_df[predict_cols]

In [35]:
scores_df = pd.DataFrame(y_pred, columns=['receiving_yards','receiving_tds','receptions'])
df_predictions = pd.concat([df_predictions, scores_df], axis=1)

In [36]:
df_predictions.head(50)
df_predictions.to_excel('wr_model_four.xlsx', index=True)

In [37]:
from flask import Flask, render_template
app = Flask(__name__)

@app.route('/')
def display_dataframe():
    return df_predictions.to_html()

if __name__ == '__main__':
    app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Nov/2023 11:05:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Nov/2023 11:05:48] "GET /favicon.ico HTTP/1.1" 404 -
